In [5]:
from os import listdir
from fragment_file_reader import fragment_file_reader

# What directory are the text files stored in?
basedir = '/Users/fraser/Github/MagneticFragmentation/fragment_properties/'

# Get list of files to use
files = [basedir + i for i in listdir(basedir)]

# Split the list into two lists, one for positive fragments and one for negative fragments
negfiles = [s for s in files if 'n.' in s]
posfiles = [s for s in files if 'p.' in s]

# Set up the individual store dictionaries
negstore = {}
negstore_number = 0
posstore = {}
posstore_number = 0

# For the first image, there is no previous timestep to compare against. All fragments are put into the store.
[negdates, neglatitudes, neglongitudes] = fragment_file_reader(negfiles[0])

for i in range(len(negdates)):
    negstore[str(negstore_number)] = [negdates[i]], [neglatitudes[i]], [neglongitudes[i]]
    negstore_number += 1

[posdates, poslatitudes, poslongitudes] = fragment_file_reader(posfiles[0])

for i in range(len(posdates)):
    posstore[str(posstore_number)] = [posdates[i]], [poslatitudes[i]], [poslongitudes[i]]
    posstore_number += 1

{'64': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [582.5], [177.0]), '73': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [570.4], [205.2]), '20': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [610.9], [90.8]), '12': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [585.9], [80.9]), '40': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [543.9], [124.4]), '82': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [329.9], [314.6]), '94': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [281.1], [398.1]), '60': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [587.0], [165.5]), '7': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [584.1], [67.3]), '47': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [598.0], [130.5]), '74': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [595.0], [205.0]), '36': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [549.3], [123.7]), '87': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 30000